In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import  cross_val_score
from scipy.stats import randint
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge

# 读取数据
#记得修改绝对路径
file_path = "C:\\Users\\15271\\Desktop\\量化\\数据获取+清理\\Preprocessing code\\merged_data.csv"  # 确保路径正确
df = pd.read_csv(file_path)

# 按 `FileName` 分组分析
factor_importance_results = {}
num_top_factors = 10  # 选择前 X 个重要因子


group = 
name = 
feature_cols=
X,y

In [75]:
#def data_classify(group, name):
    #group = group.sort_values(by = "Date")

    # 计算目标变量：未来90天收益率
    #group["future_30d_return"] = (group["Close"].shift(-30) - group["Close"]) / group["Close"]
    #group = group.dropna(subset=["future_30d_return"])  # 移除缺失目标值的行
    
    #group.set_index('Date', inplace=True)
    # 计算滚动平均
    #window_size = 30
    #group["future_30d_return"] = group["Close"].rolling(window=window_size).mean()
    #group = group.dropna(subset=["future_30d_return"])  # 移除缺失目标值的行
    
    
     # 处理数据：填充缺失值 & 归一化
    #pipeline = Pipeline([
       # ('imputer', SimpleImputer(strategy='mean')),
        #('scaler', StandardScaler())
      #  ])
   # print(type(pipeline))
    #SimpleImputer(strategy='mean'): 
    # This step replaces missing values in your data with the mean of the respective column
    
    #StandardScaler(): 
    # This step standardizes features by removing the mean and scaling to unit variance.
    
    #selecting feature columns
    #feature_cols = [col for col in group.columns if col not in ["FileName", "Date", "future_30d_return"]]
      
    #处理数据：填充缺失值 & 归一化
    #X = pipeline.fit_transform(group[feature_cols])
   # y = group["future_30d_return"].values
    
    # 确保 `feature_cols` 与 `X` 长度一致
    #feature_cols = group[feature_cols].columns.tolist()
    #print(group.shape[1])
    #return X, y, feature_cols
    

In [76]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def data_classify(group, name):
    group = group.sort_values(by="Date")

    # 计算未来30天收益率
    group["future_30d_return"] = (group["Close"].shift(-30) - group["Close"]) / group["Close"]
    group = group.dropna(subset=["future_30d_return"])  # 移除缺失目标值的行
    
    # 设置索引为日期
    group.set_index('Date', inplace=True)

    # 创建特征和目标变量
    window_size = 30
    features = []
    targets = []

    for i in range(len(group) - window_size - 30):  # 留出最后30天进行预测
        X = group.iloc[i:i + window_size]["Close"].values  # 提取窗口内的收盘价
        y = group["future_30d_return"].iloc[i + window_size]  # 目标是窗口结束后的收益率
        features.append(X)
        targets.append(y)

    # 转换为DataFrame
    X = pd.DataFrame(features)
    y = pd.Series(targets)

    # 处理数据：填充缺失值 & 归一化
    pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])

    # 处理数据：填充缺失值 & 归一化
    X = pipeline.fit_transform(X)
    
    # 确保特征列与 X 长度一致
    feature_cols = [f"Feature_{i}" for i in range(X.shape[1])]
    
    print("Features shape:", X.shape)
    print("Targets shape:", y.shape)
    
    return X, y, feature_cols

partameter distribution for a hyperparameter tuning process

    n_estimators: The number of trees in the forest

    max_depth: The maximum depth of each tree, controlliing how deep the trees can grow

    min_sample_split: The minimum number of samples required to split and internal node.

    min_samples_leaf:The minimum nuber of samples required to be at a leaf node.

RandomForestRegressor():
    n_estimators: int, default = 100

    criterion: str, default = "squared_error", 也可以是"absolute_error","poisson"泊松回归（仅用于非负目标变量）
        poisson: ???
    
    



In [77]:
def cross_ (X_train, y_train, best_rf):  #参数：交叉验证折数
    cv_folds = 5
    scoring = "r2"
    
    #交叉检验，评估模型性能
    scores = cross_val_score(best_rf, X_train, y_train, cv=cv_folds, scoring=scoring)  
    
    return scores

In [78]:

def ridge(X_train, y_train, model):
    #规范正则化强度
    alpha = 0.1
    
    model = Ridge(alpha = alpha)  # alpha 是正则化强度
    model.fit(X_train, y_train)
    
    return model

In [79]:

def testing(X_test, y_test, best_rf):
# 使用测试集进行预测
    y_pred = best_rf.predict(X_test)

    # 计算均方误差 (MSE)
    mse = mean_squared_error(y_test, y_pred)

    # 计算 R²（决定系数）
    r2 = r2_score(y_test, y_pred)

    print(f"Mean Squared Error (MSE) on Test Set: {mse}")
    print(f"R² Score on Test Set: {r2}")
    
    return r2, mse


In [80]:
def training(X, y, feature_cols):
     # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # partameter distribution for a hyperparameter tuning process
    param_dist = {
        "n_estimators": randint(200, 300),
        "max_depth": randint(10, 20),
        "min_samples_split": randint(9, 11),
        "min_samples_leaf": randint(1,4)
    }
    
    #Creating a randon forest model
    rf = RandomForestRegressor(random_state=42,max_features='sqrt')
    
    #Trying to optimize 超参数，随机搜索以优化随机森林模型超参数，相当于这只是一个model，但是需要后续输入数据进行训练
    search = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=20, cv=7, n_jobs=-1, random_state=42)
    #Training the model (这里相当于把数据输进去进行训练)
    search.fit(X_train, y_train)
    
    
    # 使用最佳参数训练最终模型
    best_rf = search.best_estimator_
      #这里找到最佳超参数组合，此时这个模型已经经过交叉验证
      #但是这个是基于折叠出来的部分数据上进行训练的，因此没有在整个数据集上面进行训练
      #所以下面这一步就是要再在整个数据集上面再训练一次
    best_rf.fit(X_train, y_train)
    
    
    
      # 计算因子重要性
    processed_feature_cols = feature_cols[:X.shape[1]]  # 除了return以外的所有值
    
      #X.shape[1]返回的是特征矩阵的列数，
      #如果X = np.array([1,2],[3,4]), feature_cols = ["f1", "f2", "f3", "f4"]
      #X.shape == np.array(2,2)
      # feature_cols[ : X.shape[1]] 返回的是第一列和第二列
      #X.shape[1]可以让我们直接选取X的所有列
    
    #获取特征重要性（各个特征对模型预测结果的贡献程度）
    #importance 返回的是numpy数组
    importance = best_rf.feature_importances_
    
    #嵌套dataframe
    importance_df = pd.DataFrame({'Factor': processed_feature_cols, 'Importance': importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)
    
    ##交叉检验
    #参数：交叉验证折数：
    cv_folds = 5
    scoring = "r2"
    
    #交叉检验，评估模型性能
    scores = cross_val_score(best_rf, X_train, y_train, cv=cv_folds, scoring=scoring) 
     
    return importance_df, best_rf, scores, X_train, X_test, y_train, y_test
    
    
    

过拟合：训练数据上表现得好，但是test上performance差，捕捉到训练数据上的噪声和细节，但没学习到普遍规律

欠拟合：表现都不好

In [81]:
factors = pd.DataFrame(columns= ["FileName", "Best_rf", " Importance_dataframe", "Parameter", "training_R2.mean", "testing_R2"])

for name, group in df.groupby("FileName"):
  #Data modification
  X, y, feature_cols = data_classify(group, name)
 
  #training
  importance_df, best_rf, scores, X_train, X_test, y_train, y_test = training(X, y , feature_cols)
  print("The score of the model is ", scores)
 
  #testing
  r2, mse = testing(X_test, y_test, best_rf)
  
 # while (True):
   # if mse > 0.005 and scores.mean() > 0.6:
      #break
    #else: 
      #正则化弱化噪声因子
     # best_rf = ridge(X_train, y_train, best_rf)
      
     
     # scores = cross_ (X_train, y_train, best_rf)
      #重新test 新的model
      #r2, mse = testing(X_test, y_test, best_rf)
      
  #生成储存超参数的dataframe
  parameter = pd.DataFrame(columns=["n_estimators", "max_depth", "min_samples_split", "min_samples_leaf"])
  parameter.loc[len(parameter)] = {"n_estimators": best_rf.n_estimators, "max_depth": best_rf.max_depth, "min_samples_split": best_rf.min_samples_split, "min_samples_leaf": best_rf.min_samples_leaf}
  #重新挑选因子
    # 计算因子重要性
  processed_feature_cols = feature_cols[:X.shape[1]]  # 除了return以外的所有值
    
    #X.shape[1]返回的是特征矩阵的列数，
    #如果X = np.array([1,2],[3,4]), feature_cols = ["f1", "f2", "f3", "f4"]
    #X.shape == np.array(2,2)
    # feature_cols[ : X.shape[1]] 返回的是第一列和第二列
    #X.shape[1]可以让我们直接选取X的所有列
    
    #获取特征重要性（各个特征对模型预测结果的贡献程度）
    #importance 返回的是numpy数组
  importance = best_rf.feature_importances_
    
    #嵌套dataframe
  importance_df = pd.DataFrame({'Factor': processed_feature_cols, 'Importance': importance})
  importance_df = importance_df.sort_values(by='Importance', ascending=False)
    
  factors.loc[len(factors)] = {"FileName": name, "Best_rf":best_rf, " Importance_dataframe": importance_df, "Parameter": parameter, "training_R2.mean": scores.mean(), "testing_R2": r2}
  #print(factors.iloc[-1])

Features shape: (465, 30)
Targets shape: (465,)
The score of the model is  [0.52452124 0.56775339 0.64493342 0.53989183 0.3987508 ]
Mean Squared Error (MSE) on Test Set: 0.00370726789381344
R² Score on Test Set: 0.5372160457830729
Features shape: (465, 30)
Targets shape: (465,)


KeyboardInterrupt: 

In [ ]:
factors.info()

#用pickle导出dataframe

factors.to_pickle("未来三十天随机森林(1).pkl")


